# Compress Images 🗜

This utlility notebook takes all high-res plant leaves photos from the source directory, compresses them and then writes them to a new directory. Both source and destination directory have the exact same paths and filenames. Any compression setting can be used to write a new directory of compressed images. 

<details open>
<summary> Benchmark </summary>
    
<pre><code class="lang-yaml"><span class="hljs-symbol">Benchmark:</span> <span class="hljs-number">4</span> vCPUs, <span class="hljs-number">16</span> GB vRAM Amazon EC2 Linux
Run <span class="hljs-number">1</span>:
<span class="hljs-symbol">    Photos:</span> ~ <span class="hljs-number">4</span>,<span class="hljs-number">500</span>
<span class="hljs-symbol">    Time:</span> ~ <span class="hljs-number">6</span> mins
<span class="hljs-symbol">    Threads:</span> <span class="hljs-number">10</span>

Run <span class="hljs-number">2</span>:
<span class="hljs-symbol">    Photos:</span> ~ <span class="hljs-number">4</span>,<span class="hljs-number">500</span>
<span class="hljs-symbol">    Time:</span> ~ <span class="hljs-number">5</span> mins
<span class="hljs-symbol">    Threads:</span> <span class="hljs-number">20</span>
</code></pre>

</details>

Import necessary libraries

In [1]:
import os
import pathlib
import shutil
import sys
from concurrent.futures import ThreadPoolExecutor

from PIL import Image
from tqdm.autonotebook import tqdm

tqdm.pandas()

/tmp/ipykernel_18049/2744342392.py:10: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Create OS-agnostic paths

In [2]:
HD_IMG_DIR: str = "Plants_2"
CMP_IMG_DIR: str = "Plants_2_compressed"
COMPRESS_SIZE: tuple[int, int] = (200, 200)

# store the filepath address of the current file
__file__: str = os.path.abspath("")

ROOT_DIR_PATH: str = os.path.join(pathlib.Path(__file__).parent.parent)
HD_IMG_DIR_PATH: str = os.path.join(ROOT_DIR_PATH, "data", HD_IMG_DIR)
CMP_IMG_DIR_PATH: str = os.path.join(ROOT_DIR_PATH, "data", CMP_IMG_DIR)

Create commpressed image directory

In [11]:
# delete compressed root directory if exists
if os.path.exists(CMP_IMG_DIR_PATH) and os.path.isdir(CMP_IMG_DIR_PATH):
    shutil.rmtree(CMP_IMG_DIR_PATH)

os.makedirs(CMP_IMG_DIR_PATH)

Create a hashmap of source directory files to destination directory files. Create all required destination directories

In [12]:
image_compress_map: dict[str, str] = {}

# do a depth-first-search based walk through the directory tree
for path, directories, files in os.walk(HD_IMG_DIR_PATH):

    # if leaf nodes are detected
    if files:

        # iterate through each file
        for img_file in files:

            # get hd and compressed file paths
            hd_file_abs_path: str = os.path.join(path, img_file)
            cmp_file_abs_path: str = hd_file_abs_path.replace(HD_IMG_DIR, CMP_IMG_DIR)

            # create directory path
            cmp_folder_abs_path: str = os.path.dirname(cmp_file_abs_path)
            # create compressed subdirectory (idempotent)
            os.makedirs(cmp_folder_abs_path, exist_ok=True)

            # add hd to compressed file mapping
            image_compress_map[hd_file_abs_path] = cmp_file_abs_path

Concurrently write multiple images after compressing to destination

In [13]:
# create a tuple version of image compress mapping dictionary for function argument passing
image_compress_map_tuples: list[tuple[str, str]] = [
    (k, v) for k, v in image_compress_map.items()
]

# create a progress bar to track progress across threads
progress_bar = tqdm(total=len(image_compress_map.keys()), position=0, leave=True)

# thread-safe function
def compress_image(source_path: str, destination_path: str):
    hd_img: Image.Image = Image.open(source_path)
    cmp_img: Image.Image = hd_img.resize(COMPRESS_SIZE)
    cmp_img.save(destination_path, "JPEG", quality=100)
    progress_bar.update(1)


# concurrently compress multiple images and save them
with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(
        lambda path_tuple: compress_image(*path_tuple), image_compress_map_tuples
    )

100%|██████████| 4502/4502 [05:00<00:00, 19.84it/s]